In [2]:
import pandas as pd

In [11]:
filepath = "/workspace/qwen-3-32b/dynamics/embedding/pc1_deltas.parquet"
embeddings_path = "/workspace/gemma-2-27b/dynamics/embedding/user_turns/shard-00000.parquet"

In [12]:
df = pd.read_parquet(filepath)
df.head()

,short_model,short_auditor_model,model,auditor_model,domain,persona_id,topic_id,response_id,prev_pc1,next_pc1,pc1_delta
0,qwen-3-32b,gpt-5,Qwen/Qwen3-32B,openai/gpt-5-chat,coding,0,0,2,11.248988,12.122361,0.873373
1,qwen-3-32b,gpt-5,Qwen/Qwen3-32B,openai/gpt-5-chat,coding,0,0,4,12.122361,9.710284,-2.412077
2,qwen-3-32b,gpt-5,Qwen/Qwen3-32B,openai/gpt-5-chat,coding,0,0,6,9.710284,9.222891,-0.487393
3,qwen-3-32b,gpt-5,Qwen/Qwen3-32B,openai/gpt-5-chat,coding,0,0,8,9.222891,10.911038,1.688148
4,qwen-3-32b,gpt-5,Qwen/Qwen3-32B,openai/gpt-5-chat,coding,0,0,10,10.911038,10.651533,-0.259505


In [13]:
df = pd.read_parquet(embeddings_path)
df.head()

# print length of vector in embedding field
print(len(df.iloc[0]['embedding']))

# print first 100 characters of text field
print(df.iloc[0]['text'])



1024
I think my Metropolis-Hastings code is borked because every proposal gets accepted. How can the acceptance ratio end up 1 if my proposal distribution is symmetric?
